In [ ]:
import joblib
import sqlite3
import pandas as pd
from collections import Counter
from surprise import SVD
from sklearn.metrics.pairwise import cosine_similarity

# Conectar a base de datos
conn = sqlite3.connect("db/market_place.db")

# Cargar reglas de asociación (Apriori)
rules = joblib.load("models/MBA/MBAreglas_apriori.joblib")

# Cargar datos de productos y vectorizador NLP
productos = pd.read_sql_query("SELECT product_id, product_name FROM products", conn)
tfidf_vectorizer = joblib.load("models/NLP/tfidf_vectorizer.joblib")
tfidf_matrix = joblib.load("models/NLP/tfidf_matrix.joblib")

# Cargar segmentos
usuarios = pd.read_sql_query("SELECT user_id, segmento FROM users", conn)
segmentos_modelos = {
    segmento: joblib.load(f"models/SVD/SVD_{segmento}.joblib")
    for segmento in usuarios['segmento'].unique()
}


def get_segmento_usuario(user_id):
    row = usuarios[usuarios['user_id'] == user_id]
    if not row.empty:
        return row.iloc[0]['segmento']
    return None


def get_productos_no_comprados(user_id):
    query = """
    SELECT DISTINCT product_id FROM products
    WHERE product_id NOT IN (
        SELECT product_id FROM orders WHERE user_id = ?
    )
    """
    df = pd.read_sql_query(query, conn, params=(user_id,))
    return df['product_id'].tolist()


def recomendaciones_svd(user_id, top_n=10):
    segmento = get_segmento_usuario(user_id)
    model = segmentos_modelos.get(segmento)
    if not model:
        return []

    productos = get_productos_no_comprados(user_id)
    predicciones = [(pid, model.predict(user_id, pid).est) for pid in productos]
    recomendaciones = sorted(predicciones, key=lambda x: x[1], reverse=True)[:top_n]
    return [pid for pid, _ in recomendaciones]


def recomendaciones_mba(productos_en_carrito):
    recomendados = set()
    for producto in productos_en_carrito:
        sub_rules = rules[rules['antecedents'].apply(lambda x: producto in x)]
        recomendados.update(sub_rules['consequents'].explode().tolist())
    return list(recomendados)


def recomendaciones_nlp(query, top_n=10):
    vector = tfidf_vectorizer.transform([query])
    similitudes = cosine_similarity(vector, tfidf_matrix).flatten()
    indices = similitudes.argsort()[::-1][:top_n]
    return productos.iloc[indices]['product_id'].tolist()


def recomendar_para_usuario(user_id, productos_en_carrito=[], busqueda=None, top_n=10):
    # Obtener recomendaciones individuales
    rec_svd = recomendaciones_svd(user_id, top_n)
    rec_mba = recomendaciones_mba(productos_en_carrito)
    rec_nlp = recomendaciones_nlp(busqueda, top_n) if busqueda else []

    # Ponderación (puedes ajustar los pesos)
    pesos = {'svd': 0.5, 'mba': 0.3, 'nlp': 0.2}
    contador = Counter()

    for p in rec_svd:
        contador[p] += pesos['svd']
    for p in rec_mba:
        contador[p] += pesos['mba']
    for p in rec_nlp:
        contador[p] += pesos['nlp']

    # Top productos finales
    top_productos = [prod for prod, _ in contador.most_common(top_n)]
    return top_productos


# ▶️ Ejemplo de uso
if __name__ == "__main__":
    user = 1
    carrito = [196, 14084]  # productos en el carrito
    busqueda = "chocolate sin azúcar"  # consulta del usuario

    recomendaciones = recomendar_para_usuario(user, carrito, busqueda)
    nombres = productos[productos['product_id'].isin(recomendaciones)]
    print("\n🎯 Recomendaciones finales:")
    for nombre in nombres['product_name']:
        print("👉", nombre)
